In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from gears import PertData, GEARS
import numpy as np
import pandas as pd
from tqdm import tqdm
import scanpy as sc
import anndata
from scipy.sparse import csr_matrix

In [3]:
data_dir = '/home/vinas/sc_perturbation_benchmark/data/tian_CRISPRa_2021_scperturb'

#### Download data

Download data from:
```
https://zenodo.org/records/7041849/files/TianKampmann2021_CRISPRa.h5ad?download=1
```

#### Process data

In [4]:
adata = anndata.read_h5ad(f'{data_dir}/TianKampmann2021_CRISPRa.h5ad')

In [6]:
adata

AnnData object with n_obs × n_vars = 21193 × 33538
    obs: 'guide_id', 'perturbation', 'tissue_type', 'celltype', 'cancer', 'disease', 'perturbation_type', 'organism', 'batch', 'nperts', 'ngenes', 'ncounts', 'percent_mito', 'percent_ribo', 'percent_hemo'
    var: 'ensembl_id', 'ncounts', 'ncells'

In [7]:
genes = adata.obs['perturbation'].unique().tolist()

for i, gene in enumerate(genes):
    if gene not in adata.var_names.tolist():
        print(gene)

control
ATP5C1
SMEK1


In [11]:
target_genes = adata.obs['perturbation'].tolist()
target_genes[target_genes=='ATP5C1']='ATP5F1C'
target_genes[target_genes=='SMEK1']='PPP4R3A'
adata.obs['perturbation'] = target_genes

In [12]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=5000, subset=False)

In [14]:
adata.var['gene_name'] = adata.var.index
adata.obs['gene'] = adata.obs['perturbation']
hvg_flag_ = adata.var['highly_variable'].values
gene_flag_ = adata.var['gene_name'].isin(adata.obs['gene'].values).values
select_flag_ = np.logical_or(hvg_flag_, gene_flag_)
condition_flag_ = adata.obs['gene'].isin(adata.var['gene_name'].values.tolist()+['control']).values
adata_subset = adata[condition_flag_, select_flag_]

In [15]:
pert_counts = adata_subset.obs['gene'].value_counts()
pert_counts

gene
TTBK2      592
INSR       494
NOP56      467
control    434
RRAS2      422
          ... 
CIC         67
SLC4A1      45
SPI1        40
HRK         37
PPP4R3A      1
Name: count, Length: 100, dtype: int64

In [16]:
# Discard ATP5PD pert (GEARS needs > 1 sample to calculate statistics)
m = pert_counts > 1
keep_perts = m[m].index.values
adata_subset = adata_subset[adata_subset.obs['gene'].isin(keep_perts)]

In [17]:
# Prepare data in GEARS format
adata_subset.obs['condition'] = [i+'+ctrl' for i in adata_subset.obs['gene'].values]
adata_subset.obs['condition'] = adata_subset.obs['condition'].replace({'control+ctrl': 'ctrl'})
adata_subset.obs['cell_type'] = 'iPSC'
len(adata_subset.obs['condition'].unique())

99

In [18]:
out_dir = f'{data_dir}'
pert_data = PertData(out_dir)
pert_data.new_data_process(dataset_name = 'CRISPRa', adata = adata_subset)

Downloading...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9.46M/9.46M [00:01<00:00, 7.67MiB/s]
Downloading...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 559k/559k [00:00<00:00, 1.41MiB/s]
Creating pyg object for each cell in the data...
Creating dataset file...
 27%|█████████████████████████████████████████████████████████████████████████                                                                                                                                                                                                   | 27/99 [04

TUSC1+ctrl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [13:03<00:00,  7.91s/it]
Done!
Saving new dataset pyg object at /home/vinas/sc_perturbation_benchmark/data/tian_CRISPRa_2021_scperturb/crispri/data_pyg/cell_graphs.pkl
Done!
